<a href="https://colab.research.google.com/github/sanjayathreya/cs598dl4h-project/blob/main/src/Descriptive-Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Reproducibility summary**

This notebook describes the methods followed by the paper **Chang Lu, Tian Han, and Yue Ning. 2021a. Context- aware health event prediction via transition functions on dynamic disease graphs(Chet). ArXiv, abs/2112.05195**

1.   **Claim 1**: For heart failure prediction task, Chet outperforms the baseline models based on metrics such as **AUC and F1-scores on MIMIC III and MIMIC IV data sets.**

2. **Claim 2**: For diagnosis prediction task, Chet outperforms the baseline models based on MIMIC III and MIMIC IV data sets. The authors compare w-F1 is a weighted sum of F1 scores for all medical codes and R@k which is an average ratio of desired medical codes in top k predictions by the total number of desired medical codes in each visit.

To verify these claims, we reproduced these results MIMIC III- carevue (Johnson et al., 2022) which excludes overlap of patients in MIMIC IV, and MIMIC IV (Johnson et al., 2023). Additionally, we investigated the effectiveness of the model under different experimental setups such comparing performance of model by changing the number of training epochs, using a different pre-processing
method to extract data, ablation studies that do not include dynamic graph and transition functions.

***This notebook describes the methods followed on reproducing the paper on [MIMIC III carevue dataset](https://physionet.org/content/mimic3-carevue/1.4/)***








# Initial setup

In [1]:
!git clone https://github.com/sanjayathreya/cs598dl4h-project
!mv /content/cs598dl4h-project /content/CHET

Cloning into 'cs598dl4h-project'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 326 (delta 0), reused 1 (delta 0), pack-reused 321
Receiving objects: 100% (326/326), 14.61 MiB | 7.24 MiB/s, done.
Resolving deltas: 100% (180/180), done.
Updating files: 100% (78/78), done.


In [2]:
#@title Copy the files paitients.csv, admissions.csv and diagnoses_icd.csv to mimic3
from google.colab import drive
drive.mount('/content/drive')
!cp -a /content/drive/MyDrive/CHET/data/mimic3/raw/ /content/CHET/data/mimic3/
!cp -a /content/drive/MyDrive/CHET/data/mimic4/raw/ /content/CHET/data/mimic4/

Mounted at /content/drive


In [3]:
%cd /content/CHET/

/content/CHET


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 45.1 MB/s eta 0:00:00


Change working directory to CHET/src

In [5]:
%cd /content/CHET/src
%pwd

/content/CHET/src


'/content/CHET/src'

# Preprocessing and dataset statistics

In order to preprocess MIMIC4dataset we need to copy over the mapping of ICD10CM to ICD9CM to the cache folder on the containerized colab environment.

In [6]:
from pyhealth.medcode import CrossMap
from pyhealth.datasets import MIMIC4Dataset,MIMIC3Dataset
!cp /content/CHET/ICD10CM_to_ICD9CM.csv /root/.cache/pyhealth/medcode

Create parsed datasets for MIMIC III Carevue subset and 


In [21]:
from preprocess import *
import os
seed = 2000
dataset_name = 'mimic3'
data_path = os.path.join('..','testing')
dataset = dataset_name
dataset_path = os.path.join(data_path, dataset)
parsed_sample_path = os.path.join(dataset_path, 'parsed')
parsed_main_path = os.path.join(data_path, dataset, 'parsed')
encoded_path = os.path.join(dataset_path, 'encoded')
standard_path = os.path.join(dataset_path, 'standard')


patient_admission, admission_codes, ds = generate_parsed_datesets(dataset,parsed_main_path,ret_value=True)
df = get_stats_df(patient_admission, admission_codes)
df

,# patients,# maximum visits,average # visits,max # diagnoses codes,avg # diagnoses codes
0,2169,23,2.447211,39,10.702148


# Preprocessing Tasks

**Step 1**: Use Tokenizer object in pyhealth to encode admission codes

In [13]:
  max_admission_num = get_stats(patient_admission, admission_codes)

  # Generate code map and encode admission_codes
  codes = list(admission_codes.values())
  codes = list(set(flatten_list(codes)))
  tokenizer = Tokenizer(tokens=codes)
  code_map = tokenizer.vocabulary.token2idx
  admission_codes_encoded = { admission_id: list(set(tokenizer.convert_tokens_to_indices(codes))) for admission_id, codes in admission_codes.items() }
  code_num = len(code_map)
  print('There are %d codes' % code_num)






patient num: 2169
max admission num: 23
mean admission num: 2.45
max code num in an admission: 39
mean code num in an admission: 10.70
There are 2821 codes


**Step 2**: Generate code levels. In this step we take the diagnoses codes for all admissions encode them into a 3 level hierarchy. This hiearchy is derived using ancestoral relationship in pyhealth. Where there are more than 3 levels , we force the hierarchy to three levels. The approach is similar to the one followed by the authors of the paper.

In [17]:
# diagnosis code levels
code_levels = generate_code_levels(code_map)
print("\ncompleted code levels")
code_levels[:2]

generating code level matrix for codes: 100%|██████████| 2821/2821 [00:00<00:00, 7702.46it/s]


completed code levels


array([[ 15, 120,  88,   0],
       [ 18, 179, 818,   1]])

**Step3**: Split patients into Train , validation and test datasets. 

While performing the split , we ensure that all diagnoses codes are seen in the training set and the admission with maximum number of diagnosis codes are also included. The reason is to ensure no unseen codes in testing and ensure no out of memory exceptions during testing.

After this step we save all encoded datasets to encoded folder for further processing.

In [22]:
# Split dataset to train , validation, test 80/10/10
train_pids, valid_pids, test_pids  = split_patients(patient_admission, admission_codes, code_map, ratios=(.80,.10,.10), seed=seed)
all_pids = { 'train_pids': train_pids, 'valid_pids': valid_pids, 'test_pids': test_pids }
save_files(encoded_path, patient_admission =patient_admission, codes_encoded=admission_codes_encoded, code_map = code_map, pids = all_pids)

Split Paitents: 100%|██████████| 2821/2821 [00:00<00:00, 265009.22it/s]

Split patient_admission #: 2169 into 
	 Train #: 1735 
	 Validation #: 218 
	 Test #: 216


**Step5**: In this step, we generate the code code adjacency matrix   

If a diagnosis code pair $(c_i, c_j)$, co-occurred in patients visit then , two edges with $\overrightarrow{(i,j)}$ and $\overleftarrow{(i,j)}$  are recorded. These are asymmetric because the authors surmise that two diseases may not have equal influence on each other. $f_{ij}$ is the total co-occurrence frequency $(c_i, c_j)$ for \textbf{all} patient visits. In order to detect important disease relationships, they filter out combinations with low frequency. $\Delta_i = {c_j | 
\frac{f_{ij}}{\sum_{j=1}^{d}f_{ij}} \geq \delta}$. This is stored in adjacency matrix $\mathcal{A}$ which is static and measures global co-occurrence frequencies of diseases.

In [23]:
# generate global adjacency matrix
adj_matrix = generate_code_code_adjacent(train_pids, patient_admission, admission_codes_encoded, code_num, threshold=0.01)
norm_adj_matrix = normalize_adj(adj_matrix)
save_dict = {'code_adj': norm_adj_matrix}
save_files(standard_path, type='standard', **save_dict)

generating global code code adjacent matrix: 100%|██████████| 1735/1735 [00:00<00:00, 4022.65it/s]


NameError: ignored

In [ ]:
# generate all datasets
datasets = ['train', 'valid', 'test' ]
for idx, dataset in enumerate(datasets):
  if dataset == 'train':
    pids = train_pids
  elif dataset == 'valid':
    pids = valid_pids
  else:
    pids = test_pids
  print(f'\nprocessing standard data for {dataset}\n')
  (X, Y, visit_lens) = construct_x_y(pids, patient_admission, admission_codes_encoded, max_admission_num, code_num)
  N_t = build_neighbor_codes(X, visit_lens, adj_matrix)
  M_t = divide_disease_type_matrices(X, N_t, visit_lens)
  Y_hf = construct_hf_label_y(Y, code_map)